<a href="https://colab.research.google.com/github/s-shahpouri/DL_PET/blob/main/PET_Correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup environment

In [20]:
!python -c "import monai" || pip install -q "monai-weekly[gdown, nibabel, tqdm, ignite]"
!python -c "import matplotlib" || pip install -q matplotlib
!python -c "import pydicom" || pip install pydicom
!python -c "import dicom2nifti" ||pip install dicom2nifti



from monai.utils import first, set_determinism
from monai.transforms import (EnsureChannelFirstd, Compose, CropForegroundd, LoadImaged, Orientationd, RandCropByPosNegLabeld, ScaleIntensityRanged, Spacingd)
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset
from monai.apps import download_and_extract
from monai.transforms import CenterSpatialCropd
from monai.transforms import Resized
import torch
import matplotlib.pyplot as plt
import os
import glob
import torch.nn as nn

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'monai'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.5/266.5 kB 25.5 MB/s eta 0:00:00


## Set dataset path

In [13]:
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')

root_dir = '/content/drive/My Drive/PET_data'
zip_file_path = os.path.join(root_dir, 'pet.zip')
extracted_folder_path = os.path.join(root_dir, 'sapmle_data')

# Create the extracted data folder if it doesn't exist
os.makedirs(extracted_folder_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# List the contents of each patient folder, including subfolders
for patient_folder in os.listdir(extracted_folder_path):
    patient_folder_path = os.path.join(extracted_folder_path, patient_folder)

    # List subfolders within the patient folder
    subfolders = [subfolder for subfolder in os.listdir(patient_folder_path) if os.path.isdir(os.path.join(patient_folder_path, subfolder))]

    print(f"Contents of {patient_folder}: {subfolders}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Contents of P010: ['MAC', 'NAC']
Contents of P001: ['MAC', 'NAC']
Contents of P002: ['MAC', 'NAC']
Contents of P003: ['MAC', 'NAC']
Contents of P004: ['MAC', 'NAC']
Contents of P005: ['MAC', 'NAC']
Contents of P006: ['MAC', 'NAC']
Contents of P007: ['MAC', 'NAC']
Contents of P008: ['MAC', 'NAC']
Contents of P009: ['MAC', 'NAC']


In [21]:
from google.colab import drive
import os

import dicom2nifti

drive.mount('/content/drive')
root_dir = '/content/drive/My Drive/PET_data/sample_data'

# Get a list of all patient folders
patient_folders = sorted([f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))])

# Create a list to store data dictionaries for each patient
data_dicts = []

# Iterate through each patient folder
for patient_folder in patient_folders:
    patient_path = os.path.join(root_dir, patient_folder)

    # Check if 'NAC' and 'MAC' folders exist in the patient folder
    if 'NAC' in os.listdir(patient_path) and 'MAC' in os.listdir(patient_path):

        nac_folder = os.path.join(patient_path, 'NAC')
        mac_folder = os.path.join(patient_path, 'MAC')

        # Create a data dictionary for the patient
        data_dict = {"image": nac_folder, "target": mac_folder}
        data_dicts.append(data_dict)

# Optional: Split the data into train and validation sets
train_files, val_files = data_dicts[:-1], data_dicts[-1:]




Mounted at /content/drive


In [22]:
import os
import dicom2nifti

def dicom_to_nifti(dicom_folder, output_folder):
    print(f"Converting DICOM files from {dicom_folder} to NIfTI files in {output_folder}")
    # Convert DICOM to NIfTI
    dicom2nifti.convert_directory(dicom_folder, output_folder, compression=True, reorient=True)
    print("Conversion complete.")


In [23]:
drive.mount('/content/drive')
root_dir = '/content/drive/My Drive/PET_data/sample_data'

patient_folders = sorted([f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))])

for patient_folder in patient_folders:
    dicom_folder = root_dir + "/" + patient_folder + "/MAC"
    output_folder = root_dir + "/" + patient_folder
    dicom_to_nifti(dicom_folder, output_folder)

    dicom_folder = root_dir + "/" + patient_folder + "/NAC"
    output_folder = root_dir + "/" + patient_folder
    dicom_to_nifti(dicom_folder, output_folder)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Converting DICOM files from /content/drive/My Drive/PET_data/sample_data/P001/MAC to NIfTI files in /content/drive/My Drive/PET_data/sample_data/P001
Conversion complete.
Converting DICOM files from /content/drive/My Drive/PET_data/sample_data/P001/NAC to NIfTI files in /content/drive/My Drive/PET_data/sample_data/P001
Conversion complete.
Converting DICOM files from /content/drive/My Drive/PET_data/sample_data/P002/MAC to NIfTI files in /content/drive/My Drive/PET_data/sample_data/P002
Conversion complete.
Converting DICOM files from /content/drive/My Drive/PET_data/sample_data/P002/NAC to NIfTI files in /content/drive/My Drive/PET_data/sample_data/P002
Conversion complete.
Converting DICOM files from /content/drive/My Drive/PET_data/sample_data/P003/MAC to NIfTI files in /content/drive/My Drive/PET_data/sample_data/P003
Conversion complete.
Converting DICOM